In [62]:
# Staring import of required modules to perform Network Assessment and analysis

import pandas as pd
import numpy as np
import random
import datetime

In [63]:
# Loading the sample dataset

network_data = pd.read_csv('Lab Network Devices DataFrame.csv')
print(network_data.info())
network_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Device            416 non-null    object
 1   Device Type       416 non-null    object
 2   Vendor            410 non-null    object
 3   Model             408 non-null    object
 4   Software Version  410 non-null    object
 5   EIGRP Enabling    416 non-null    bool  
 6   ISIS Enabling     416 non-null    bool  
 7   OSPF Enabling     416 non-null    bool  
 8   BGP Enabling      416 non-null    bool  
 9   MC-LAG Enabling   416 non-null    bool  
 10  VXLAN Enabling    416 non-null    bool  
 11  NAT Enabling      416 non-null    bool  
dtypes: bool(7), object(5)
memory usage: 19.2+ KB
None


,Device,Device Type,Vendor,Model,Software Version,EIGRP Enabling,ISIS Enabling,OSPF Enabling,BGP Enabling,MC-LAG Enabling,VXLAN Enabling,NAT Enabling
0,ACI-L3Out-249,Cisco IOS Switch,Cisco,WS-C3750X-24,15.2(4)E6,False,False,False,False,False,False,False
1,ASA-AA/admin/act,Cisco ASA Firewall,NaN,ASA5510,NaN,True,False,True,False,False,False,False
2,ASA-AA/context2/act,Cisco ASA Firewall,NaN,ASA5510,NaN,False,False,False,False,False,False,False
3,ASA.Switch,Cisco IOS Switch,Cisco,3560E,15.2(HI_20170202)FLO_DSGS7,False,False,False,False,False,False,False
4,ASA@Switch,Cisco IOS Switch,Cisco,3560E,15.2(HI_20170202)FLO_DSGS7,False,False,False,False,False,False,False


In [64]:
# Clean up the dataset, removing possible blank spaces and unwanted characters from data

def strip_dataset_spaces(data):
    # Clean column names
    network_data.columns = network_data.columns.str.strip()
    
    # Clan row values and change data type to be able to operate on non boolean basis
    for column in network_data.columns:
        if network_data[column].dtype != 'object':
            network_data[column] = network_data[column].astype('object')
        elif network_data[column].dtype == 'object':
            network_data[column] = network_data[column].str.title()
            network_data[column] = network_data[column].str.strip()
    return network_data

# Call the function to clean up and convert data as intended

strip_dataset_spaces(network_data)

,Device,Device Type,Vendor,Model,Software Version,EIGRP Enabling,ISIS Enabling,OSPF Enabling,BGP Enabling,MC-LAG Enabling,VXLAN Enabling,NAT Enabling
0,Aci-L3Out-249,Cisco Ios Switch,Cisco,Ws-C3750X-24,15.2(4)E6,False,False,False,False,False,False,False
1,Asa-Aa/Admin/Act,Cisco Asa Firewall,NaN,Asa5510,NaN,True,False,True,False,False,False,False
2,Asa-Aa/Context2/Act,Cisco Asa Firewall,NaN,Asa5510,NaN,False,False,False,False,False,False,False
3,Asa.Switch,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False
4,Asa@Switch,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
411,Us-Nyc-Enterprise-Sw1,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,True,False,False,False,False
412,Us-Nyc-Production-Sw1,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,True,False,False,False,False
413,Us-Portland-R1,Cisco Router,Cisco,Cgs-Mgs-Ags,15.4(2)T4,False,False,True,True,False,False,False
414,Us-San-R1,Cisco Router,Cisco,Cgs-Mgs-Ags,15.4(2)T4,False,False,True,True,False,False,False


In [65]:
# Re printing dataFrame information to make sure the changes are in place

print(network_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 416 entries, 0 to 415
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Device            416 non-null    object
 1   Device Type       416 non-null    object
 2   Vendor            410 non-null    object
 3   Model             408 non-null    object
 4   Software Version  410 non-null    object
 5   EIGRP Enabling    416 non-null    object
 6   ISIS Enabling     416 non-null    object
 7   OSPF Enabling     416 non-null    object
 8   BGP Enabling      416 non-null    object
 9   MC-LAG Enabling   416 non-null    object
 10  VXLAN Enabling    416 non-null    object
 11  NAT Enabling      416 non-null    object
dtypes: object(12)
memory usage: 39.1+ KB
None


All columns are title type now and the rows Dtypes are now 'object'

#### "Interrogating" the Data Frame 

Trying to understand more about the network at hand:

 <span style="color:green;">How many devices are running: </span>
 *  <span style="color:purple;">EIGRP </span>
 *  <span style="color:purple;">ISIS </span>
 *  <span style="color:purple;">OSPF </span>
 *  <span style="color:purple;">BGP </span>
 *  <span style="color:purple;">MC-LAG </span>
 *  <span style="color:purple;">VXLAN </span>
 *  <span style="color:purple;">NAT </span>
 

#### Complimentary Code

Will start by adding a Python list of protocols / features that will be used as a filter, to identify column names for count mapping and printing.
We have to make sure to include protocols that may appear on other networks to make this analysis as comprehensive as possible.

For example:

<span style="color:purple;"> LISP </span> is not present on the reference network but it might be on other samples.

In [66]:
# List of Network protocols and features

run_protocols = ['ospf','eigrp','isis','bgp','mc-lag','vxlan','nat','lisp','pim','vrf']

# Creating an empty list to store dynamically created variable names, depending on running protocols

dynamic_vars = []
print(dynamic_vars)

[]


#### Renaming DataFrame Columns

We will now, replace the Column name keyword <span style="color:orange;"> "Enabling" </span> with the Keyword <span style="color:orange;"> "Enabled" </span>

In [67]:
#Creating empty Dictionary to be used as "Column Mapper"

column_mapper = {}

#Creating the loop that will iterate over column names and replace values as indicated

for column_name in network_data.columns:
    for protocol in run_protocols:
        if 'Enabling' in column_name and protocol.upper() in column_name:
            column_mapper[column_name] = f'{protocol.upper()} Enabled'


print(column_mapper)

{'EIGRP Enabling': 'EIGRP Enabled', 'ISIS Enabling': 'ISIS Enabled', 'OSPF Enabling': 'OSPF Enabled', 'BGP Enabling': 'BGP Enabled', 'MC-LAG Enabling': 'MC-LAG Enabled', 'VXLAN Enabling': 'VXLAN Enabled', 'NAT Enabling': 'NAT Enabled'}


In [68]:
# Rename Columns

network_data = network_data.rename(mapper=column_mapper, axis=1)

# Print first 5 rows to verify column renaming was successful

network_data.head()

,Device,Device Type,Vendor,Model,Software Version,EIGRP Enabled,ISIS Enabled,OSPF Enabled,BGP Enabled,MC-LAG Enabled,VXLAN Enabled,NAT Enabled
0,Aci-L3Out-249,Cisco Ios Switch,Cisco,Ws-C3750X-24,15.2(4)E6,False,False,False,False,False,False,False
1,Asa-Aa/Admin/Act,Cisco Asa Firewall,NaN,Asa5510,NaN,True,False,True,False,False,False,False
2,Asa-Aa/Context2/Act,Cisco Asa Firewall,NaN,Asa5510,NaN,False,False,False,False,False,False,False
3,Asa.Switch,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False
4,Asa@Switch,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False


Having already verified Dynamic Vars list is effectively empty ([  ] result for dynamic_vards), and having replaced the column names accordingly, will proceed to create <span style="color:orange;"> TEMP </span> variable names based on running protocols registered on the current data-set that match any of the entries on <span style="color:orange;"> "run_protocols" Python list. </span>

#### Per Protocol Count 


In [69]:
# Defining iteration to create per-protocol temporary variables and applying value count to them

for protocol in run_protocols:
    for column_name in network_data.columns:
        if protocol in column_name.lower():
            print(network_data[f'{protocol.upper()} Enabled'].value_counts())
            print()
            dynamic_vars.append(f'{protocol}_devices')
            break  # Move to the next count if a match is found

OSPF Enabled
False    223
True     193
Name: count, dtype: int64

EIGRP Enabled
False    393
True      23
Name: count, dtype: int64

ISIS Enabled
False    401
True      15
Name: count, dtype: int64

BGP Enabled
False    335
True      81
Name: count, dtype: int64

MC-LAG Enabled
False    413
True       3
Name: count, dtype: int64

VXLAN Enabled
False    412
True       4
Name: count, dtype: int64

NAT Enabled
False    400
True      16
Name: count, dtype: int64



#### Translating above counts into percentage


In [70]:
# Modifying the above iteration to reflect results in Percentages

for protocol in run_protocols:
    for column_name in network_data.columns:
        if protocol in column_name.lower():
            print(protocol.upper() + ' :' + ' ' + str(round((network_data[f'{protocol.upper()} Enabled'].value_counts()[True])/len(network_data)*100, 2)) + '%')
            print()
            break  # Move to the next count if a match is found

OSPF : 46.39%

EIGRP : 5.53%

ISIS : 3.61%

BGP : 19.47%

MC-LAG : 0.72%

VXLAN : 0.96%

NAT : 3.85%



#### Next, will look into all variants of unique Device Software Versions used by network devices

This would allow to start getting an idea of how does the Standard looks from that parspective.


In [71]:
# Calculating the number of unique Sorftware Versions used by the devices

network_data['Software Version'].nunique()

104

In [72]:
# Next will print the actual Software details

network_data['Software Version'].unique()

array(['15.2(4)E6', nan, '15.2(Hi_20170202)Flo_Dsgs7',
       'Virtual.10.09.0002', '12.2(55)Se4',
       'Ethernet Routing Switch 3550T         Hw:01       Fw:5.3.0.6   Sw:V5.3.0.004 Bn:04 (C) Avaya Networks',
       '15.0(2)Se', '15.1(4)M5', '15.1(4)M2', '12.1(22)Ea14',
       '12.1(22)Ea1', '12.2(24B)',
       'Fortigate-200F V7.2.5,Build1517,230606 (Ga.F)', 'Barracuda-Fw',
       '20.9.3.1', '15.0(2)Se7', '12.2(25)See1', '15.4(3)S2', '15.7(3)M2',
       '15.4(2)T4', '14.1R4.8', '12.2(55)Se3', '12.2(55)Se7',
       'Linux Cp_Gw1 2.6.18-92Cpx86_64 #1 Smp Tue Jan 29 04:50:27 Ist 2019 X86_64',
       'Linux Cp_Gw2 2.6.18-92Cpx86_64 #1 Smp Tue Jan 29 04:50:27 Ist 2019 X86_64',
       'Linux Cp_Ha2 2.6.18-92Cpx86_64 #1 Smp Tue Jan 29 04:50:27 Ist 2019 X86_64',
       '15.2(2)E7', '12.4(15)T12', '15.2(4)M6A', 'B-13.0.R4',
       '15.2(Cml_Nightly_20150703)Flo_Dsgs7', '12.3(26)', '15.5.2.9',
       '12.3(17B)', '7.2(0)D1(1)', '03.06.06E', '15.3(3)Jf10$',
       '12.0(7)W5(16)', '7.0(3)I4(1

In [73]:
# Print Software Version Info

network_data['Software Version'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 416 entries, 0 to 415
Series name: Software Version
Non-Null Count  Dtype 
--------------  ----- 
410 non-null    object
dtypes: object(1)
memory usage: 3.4+ KB


#### Checking Network Device Vendor, Models and Device Types Stats


In [74]:
# Print Vendor details

network_data['Vendor'].unique()

array(['Cisco', nan, 'Aruba', 'Avaya', 'Fortinet', 'Barracuda',
       'Cisco Viptela', 'Juniper', 'Checkpoint', 'Nokia', 'Extreme',
       'Arista', 'Palo Alto Networks', 'Dell', 'Opengear', '3Com', 'F5'],
      dtype=object)

In [75]:
network_data['Vendor'].nunique()

16

In [76]:
# Print Model details

network_data['Model'].unique()

array(['Ws-C3750X-24', 'Asa5510', '3560E', 'Asav', '8400',
       'Ws-C3750G-48Ts', 'Ers 5520', 'Catalyst356024Ts', 'Ws-C3560-24Ts',
       '2821', '2811', 'Ws-C2950-24', 'Ws-C3750-24Ts', '2500',
       'Fortigate-200F', '200F', nan, 'Vedge-Cloud', 'Ws-C2960-48Tt-L',
       'Asr 1002-X Router', 'Cgs-Mgs-Ags', 'Mx150', 'Ws-C3750X-48',
       'Check Point 3200', 'Ws-C2960X-48Td-L', '3725',
       'Cisco3945-Chassis', '7750 Sr12', '2621', 'Summitx440-8T', '2620',
       'N7K-C7018', 'Ws-C3850-48P', 'Air-Cap3702I-A-K9', 'Rsp2',
       'N9K-C9372Px-E', 'Air-Ct5508-K9', 'N9K-C9300V', 'Iosxrv',
       'Summitx440-L2-48T', '2611', 'Virtual Wireless Lan Controller',
       'Ws-C2960-24Tt-L', 'Me-3600X-24Ts-M', 'Asr1001-X', '7206Vxr',
       'Veos', 'Ws-C2960Xr-48Ts-I', 'Ws-C3750-24P', 'Catalyst296048Tt',
       'Ws-C3560X-24', 'Pa-Vm', 'Isr4321/K9', 'Ws-C2960-24-S',
       'Catalyst3560G24Ts', 'S50N', 'Ws-C4500X-32', 'Catalyst 29Xxstack',
       'Ws-C3750X-48P', 'N9K-C9372Tx-E', 'Asa5515', 'Dcs

In [77]:
network_data['Model'].nunique()

89

In [78]:
# Print Model details

network_data['Device Type'].unique()

array(['Cisco Ios Switch', 'Cisco Asa Firewall', 'Aruba Switch',
       'Avaya Switch', 'Cisco Router', 'Fortinet Fortigate Firewall',
       'Barracuda Firewall', 'Viptela Vedge', 'Juniper Router',
       'Checkpoint Firewall R80', 'Alcatel Lucent Service Router',
       'Extreme Switch', 'Cisco Nexus Switch', 'Lwap', 'Cisco Wlc',
       'Cisco Ios Xr', 'Arista Switch', 'Palo Alto Firewall',
       'Dell Force10 Switch', 'Cisco Catalyst Switch',
       'Opengear Infrastructure Manager', '3Com Switch',
       'F5 Load Balancer', 'Aruba Wlc', 'Cisco Wap', 'Dell Sonicwall'],
      dtype=object)

In [79]:
network_data['Device Type'].nunique()

26

#### Adding a New Column Containing a List of Running Protocols Per Device

The idea behind the this process is to, later, be able to eliminate the "True" / "False" Columns to simplify readability and yield some real state for further enhancements.

<span style="color:green;"> A new column will be added to the end of the Data Frame </span>, called "Running Protocols".

The logic to populate it, is to:

1) Loop through the previously defined Python List (run_protocols - remember it is dynamic)
2) For each protocol listes under run_protocols, if that string matches the Column Name Protocol string it adds that string to the new column

In [80]:
# Define a function to extract running protocols for each row
# Then it creates a comma separated list


def get_running_protocols(row):
    running_protocols = [protocol.upper() for protocol in run_protocols if f"{protocol.upper()} Enabled" in row.index and  row[f"{protocol.upper()} Enabled"]]
    return ', '.join(running_protocols)

# Create a new column "Running Protocols" with the running protocols for each row
network_data['Running Protocols'] = network_data.apply(get_running_protocols, axis=1)

network_data.head(15)

,Device,Device Type,Vendor,Model,Software Version,EIGRP Enabled,ISIS Enabled,OSPF Enabled,BGP Enabled,MC-LAG Enabled,VXLAN Enabled,NAT Enabled,Running Protocols
0,Aci-L3Out-249,Cisco Ios Switch,Cisco,Ws-C3750X-24,15.2(4)E6,False,False,False,False,False,False,False,
1,Asa-Aa/Admin/Act,Cisco Asa Firewall,NaN,Asa5510,NaN,True,False,True,False,False,False,False,"OSPF, EIGRP"
2,Asa-Aa/Context2/Act,Cisco Asa Firewall,NaN,Asa5510,NaN,False,False,False,False,False,False,False,
3,Asa.Switch,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False,
4,Asa@Switch,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False,
5,Asa\Router,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False,
6,Asav40,Cisco Asa Firewall,NaN,Asav,NaN,False,False,False,False,False,False,False,
7,Aruba-Switch1,Aruba Switch,Aruba,8400,Virtual.10.09.0002,False,False,True,False,False,False,False,OSPF
8,Bj-3750-1,Cisco Ios Switch,Cisco,Ws-C3750G-48Ts,12.2(55)Se4,False,False,True,False,False,False,False,OSPF
9,Bj-Avaya-1,Avaya Switch,Avaya,Ers 5520,Ethernet Routing Switch 3550T Hw:01 ...,False,False,False,False,False,False,False,


#### Adding a New Column to Count and Print the number of Running Protocols per Device

Will sort the list in non-ascending order to see the devices with most running protocols first

In [81]:
# Function to count protocol occurrences
def count_protocols(protocol_str):
    if pd.isna(protocol_str) or protocol_str.strip() == '':
        return 0
    else:
        return len(protocol_str.split(', '))

# Apply the function to each row in the DataFrame
network_data['Running Protocols Count'] = network_data['Running Protocols'].apply(count_protocols)


# Verifying the modified Data Frame for the first 10 rows

network_data.head(10)

,Device,Device Type,Vendor,Model,Software Version,EIGRP Enabled,ISIS Enabled,OSPF Enabled,BGP Enabled,MC-LAG Enabled,VXLAN Enabled,NAT Enabled,Running Protocols,Running Protocols Count
0,Aci-L3Out-249,Cisco Ios Switch,Cisco,Ws-C3750X-24,15.2(4)E6,False,False,False,False,False,False,False,,0
1,Asa-Aa/Admin/Act,Cisco Asa Firewall,NaN,Asa5510,NaN,True,False,True,False,False,False,False,"OSPF, EIGRP",2
2,Asa-Aa/Context2/Act,Cisco Asa Firewall,NaN,Asa5510,NaN,False,False,False,False,False,False,False,,0
3,Asa.Switch,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False,,0
4,Asa@Switch,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False,,0
5,Asa\Router,Cisco Ios Switch,Cisco,3560E,15.2(Hi_20170202)Flo_Dsgs7,False,False,False,False,False,False,False,,0
6,Asav40,Cisco Asa Firewall,NaN,Asav,NaN,False,False,False,False,False,False,False,,0
7,Aruba-Switch1,Aruba Switch,Aruba,8400,Virtual.10.09.0002,False,False,True,False,False,False,False,OSPF,1
8,Bj-3750-1,Cisco Ios Switch,Cisco,Ws-C3750G-48Ts,12.2(55)Se4,False,False,True,False,False,False,False,OSPF,1
9,Bj-Avaya-1,Avaya Switch,Avaya,Ers 5520,Ethernet Routing Switch 3550T Hw:01 ...,False,False,False,False,False,False,False,,0


In [83]:
# Sort Values by Protocol Count

network_data.sort_values(by='Running Protocols Count', ascending=False)

,Device,Device Type,Vendor,Model,Software Version,EIGRP Enabled,ISIS Enabled,OSPF Enabled,BGP Enabled,MC-LAG Enabled,VXLAN Enabled,NAT Enabled,Running Protocols,Running Protocols Count
295,Pe-3600X-02,Cisco Router,Cisco,Me-3600X-24Ts-M,15.4(2)S,True,True,True,True,False,False,False,"OSPF, EIGRP, ISIS, BGP",4
96,Nb-Vxlan-Spine-1,Cisco Nexus Switch,Cisco,N9K-C9300V,9.3(11),False,False,True,True,False,True,False,"OSPF, BGP, VXLAN",3
95,Nb-Vxlan-Leaf-1,Cisco Nexus Switch,Cisco,N9K-C9300V,9.3(11),False,False,True,True,False,True,False,"OSPF, BGP, VXLAN",3
137,Pe-Asr1K-02,Cisco Router,Cisco,Asr1001-X,03.12.00A.S,False,True,True,True,False,False,False,"OSPF, ISIS, BGP",3
136,Pe-Asr1K-01,Cisco Router,Cisco,Asr1001-X,16.09.04,False,True,True,True,False,False,False,"OSPF, ISIS, BGP",3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
263,Nb-1502-07,Lwap,Cisco,Air-Cap3702I-A-K9,15.3(3)Jf10$,False,False,False,False,False,False,False,,0
264,Nb-1502-08,Lwap,Cisco,Air-Cap3702I-A-K9,15.3(3)Jf10$,False,False,False,False,False,False,False,,0
265,Nb-1502-10,Lwap,Cisco,Air-Cap3702I-A-K9,15.3(3)Jf10$,False,False,False,False,False,False,False,,0
266,Nb-1502-11,Lwap,Cisco,Air-Cap3702I-A-K9,15.3(3)Jf10$,False,False,False,False,False,False,False,,0


In [86]:
# Count number of devices running protocls

network_data['Running Protocols Count'].value_counts()

Running Protocols Count
0    184
1    142
2     78
3     11
4      1
Name: count, dtype: int64